In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [50]:
url = 'https://loyolagreyhounds.com/sports/womens-basketball/stats/2022-23'
response = requests.get(url)

if response.status_code == 200:
    html_content = response.content
    print("The website has been successfully brought.")
else:
    print("Failed to fetch the website.")

The website has been successfully brought.


In [22]:
soup = BeautifulSoup(html_content,'html')
main = soup.find('main',class_='main-content-placeholder')
overall = main.find('section',id='individual-overall')
table = overall.find('table','sidearm-table')
table_head = table.find('thead')
headers_label = table_head.find_all('tr')[0]
headers_column = table_head.find_all('tr')[1]

In [124]:
dict_colspan = {}
index = 0
colspan_headers = headers_label.find_all("th")

for header in colspan_headers:
    colspan_value = int(header.get("colspan", 1)) #if header dont have a colspan, colspan_value get 1 - 
    header_text = header.get_text()
    print("Column Group Header:", header_text, "- Colspan Value:", colspan_value)
    dict_colspan[header_text] = colspan_value
    
data_headers = headers_column.find_all('th')
print([data.text.strip() for data in data_headers])    

Column Group Header: # - Colspan Value: 1
Column Group Header: Player - Colspan Value: 1
Column Group Header: GP - Colspan Value: 1
Column Group Header: GS - Colspan Value: 1
Column Group Header: Minutes - Colspan Value: 2
Column Group Header: FG - Colspan Value: 3
Column Group Header: 3PT - Colspan Value: 3
Column Group Header: FT - Colspan Value: 3
Column Group Header: Scoring - Colspan Value: 2
Column Group Header: Rebounds - Colspan Value: 4
Column Group Header: PF - Colspan Value: 1
Column Group Header: AST - Colspan Value: 1
Column Group Header: TO - Colspan Value: 1
Column Group Header: STL - Colspan Value: 1
Column Group Header: BLK - Colspan Value: 1
Column Group Header: Bio Link - Colspan Value: 1
['TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FTA', 'FT%', 'PTS', 'AVG', 'OFF', 'DEF', 'TOT', 'AVG']


In [120]:
index = 0
columns_overall = []

for span,text in zip(dict_colspan.values(),dict_colspan.keys()):
    data = []
    
    if(span <= 1):
        columns_overall.append(text)
    else:
        data= [head.text.strip() + "_" + text for head in data_headers[index:index+span]]
        index += span
    
    columns_overall.extend(data)  # Yeni verileri ana liste içine ekler

df_overall = pd.DataFrame(columns=columns_overall)
df_overall

,#,Player,GP,GS,TOT_Minutes,AVG_Minutes,FGM_FG,FGA_FG,FG%_FG,3PT_3PT,...,OFF_Rebounds,DEF_Rebounds,TOT_Rebounds,AVG_Rebounds,PF,AST,TO,STL,BLK,Bio Link


In [140]:
data_list = []
path('https://loyolagreyhounds.com')

body = table.find('tbody')
players = body.find_all('tr')

for player in players[:-1]:
    row = player.find_all('td')
    data = [data.text.strip() for data in row[:-1]]
    
    bio = row[-1].find('a')['href']
    data.append(bio)
    data_list.append(data)

df_overall = pd.DataFrame(data_list,columns = columns_overall)
df_overall

,#,Player,GP,GS,TOT_Minutes,AVG_Minutes,FGM_FG,FGA_FG,FG%_FG,3PT_3PT,...,OFF_Rebounds,DEF_Rebounds,TOT_Rebounds,AVG_Rebounds,PF,AST,TO,STL,BLK,Bio Link
0,45,"Therien, Lex\n45 Therien, Lex",31,31,965,31.1,187,381,.491,2,...,93,251,344,11.1,105,32,103,49,7,/sports/womens-basketball/roster/lex-therien/1...
1,04,"Dixon, Taleah\n04 Dixon, Taleah",30,30,969,32.3,138,395,.349,44,...,6,50,56,1.9,65,66,76,44,1,/sports/womens-basketball/roster/taleah-dixon/...
2,05,"Newman, Devyne\n05 Newman, Devyne",20,13,497,24.9,67,170,.394,8,...,15,32,47,2.4,26,28,26,6,0,/sports/womens-basketball/roster/devyne-newman...
3,10,"Sims, Koi\n10 Sims, Koi",20,19,536,26.8,58,132,.439,0,...,72,104,176,8.8,34,34,42,17,18,/sports/womens-basketball/roster/koi-sims/11750
4,22,"Rozzi, Bri\n22 Rozzi, Bri",27,23,746,27.6,55,155,.355,20,...,13,56,69,2.6,58,60,49,16,0,/sports/womens-basketball/roster/bri-rozzi/11620
5,01,"Ratigan, Kelly\n01 Ratigan, Kelly",30,22,772,25.7,54,157,.344,36,...,12,53,65,2.2,49,35,48,10,1,/sports/womens-basketball/roster/kelly-ratigan...
6,13,"Tañedo, Caramina\n13 Tañedo, Caramina",29,5,509,17.6,19,69,.275,14,...,23,26,49,1.7,21,18,26,21,1,/sports/womens-basketball/roster/caramina-ta-n...
7,02,"Van-Otoo, Meliah\n02 Van-Otoo, Meliah",24,4,275,11.5,17,45,.378,2,...,7,16,23,1.0,35,22,34,10,2,/sports/womens-basketball/roster/meliah-van-ot...
8,15,"Salmerón, Laura\n15 Salmerón, Laura",20,4,273,13.7,12,48,.250,3,...,1,22,23,1.2,11,12,19,10,0,/sports/womens-basketball/roster/laura-salmer-...
9,12,"Garcia, Cristina\n12 Garcia, Cristina",30,4,356,11.9,12,56,.214,0,...,26,19,45,1.5,45,17,32,10,1,/sports/womens-basketball/roster/cristina-garc...
